In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
!pip install selfcheckgpt

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 3.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached

In [ ]:
!pip install openai

In [ ]:
from tqdm import tqdm
from selfcheckgpt.modeling_selfcheck import SelfCheckNLI
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_nli = SelfCheckNLI(device=device) # set device to 'cuda' if GPU is available



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


SelfCheck-NLI initialized to device cpu


In [ ]:
# Load the dr_review corpus.

from google.colab import drive
drive.mount('/content/drive')


query_path = '/content/drive/MyDrive/Colab Notebooks/result/evaluation.txt'
with open(query_path, 'r', encoding='utf-8') as file:
    query_lines = file.readlines()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import os
import openai
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
import os
from deepeval.test_case import LLMTestCase, LLMTestCaseParams
from deepeval.metrics import GEval

In [ ]:
openai.api_key=""

In [ ]:
def coherence(input,actual_output):
  test_case = LLMTestCase(input = input, actual_output = actual_output)
  coherence_metric = GEval(
      name="Readability",
      criteria="Readability - Evaluate whether the sentences are well-structured and complete, free from garbled text, meaningless repetition and punctuation errors, clear, and easy to read.",
      evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
  )

  coherence_metric.measure(test_case)
  return coherence_metric.score, coherence_metric.reason

In [ ]:
def logic(input,actual_output):
  test_case = LLMTestCase(input = input, actual_output = f"psychotherapist: {input}. Tom(patient): {actual_output}")
  coherence_metric = GEval(
      name="Logic",
      criteria="Logic - You will be given a question asked by the psychotherapist and a response from Tom (patient). Judge whether the response follow a logical progression and whether the response is relevant to the question in a coherent and logical manner.",
      evaluation_params=[LLMTestCaseParams.ACTUAL_OUTPUT],
  )

  coherence_metric.measure(test_case)
  return coherence_metric.score, coherence_metric.reason

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 4.3 MB/s eta 0:00:00


In [ ]:
loader = TextLoader("/content/drive/MyDrive/Colab Notebooks/result/RAG_362.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=0,
    length_function=len
)
docs = text_splitter.split_documents (documents)
embedding = OpenAIEmbeddings(openai_api_key = "")
library = FAISS. from_documents(docs,embedding)

In [ ]:
df2 = pd.DataFrame()
df2["model_name"]=None
df2["average_nli"]=None
df2["average_readability"]=None
df2["average_logic"]=None

In [ ]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/response_rag/"

file_names = os.listdir(folder_path)

for file_name in file_names:

  file_path  = folder_path+file_name
  with open(file_path, 'r', encoding='utf-8') as file:
      lines = file.readlines()

  df = pd.DataFrame()
  df["query"] = query_lines
  df["response"] = lines

  for index, example in tqdm(df.iterrows()):
    query = example["query"]
    query_answer = library.similarity_search("Assistant: "+query)
    print(query_answer)
    df.at[index, "context"] = query_answer[0].page_content



  for index, example in tqdm(df.iterrows()):
    sentences = [example["response"]]
    samples = [example["context"]]
    sent_scores_nli = selfcheck_nli.predict(
      sentences = sentences,                          # list of sentences
      sampled_passages = samples, # list of sampled passages
    )
    df.loc[index, "sent_scores_nli"] = ', '.join(map(str, sent_scores_nli))



  for index, example in tqdm(df.iterrows()):
    response = [example["response"]]
    query = [example["query"]]
    GEval_Readability,GEval_Readability_reason = coherence(query,response)
    df.loc[index, "GEval_Readability"] = GEval_Readability
    df.loc[index, "GEval_Readability_reason"] = GEval_Readability_reason

  for index, example in tqdm(df.iterrows()):
    response = [example["response"]]
    query = [example["query"]]
    GEval_Logic,GEval_Logic_reason = logic(query,response)
    df.loc[index, "GEval_Logic"] = GEval_Logic
    df.loc[index, "GEval_Logic_reason"] = GEval_Logic_reason

  pure_file_name = file_name[:-4]
  df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/result/{pure_file_name}.csv", index=False)

  df["sent_scores_nli"] = df["sent_scores_nli"].astype(float)
  average_nli = df["sent_scores_nli"].mean()
  df["GEval_Readability"] = df["GEval_Readability"].astype(float)
  average_readability = df["GEval_Readability"].mean()
  df["GEval_Logic"] = df["GEval_Logic"].astype(float)
  average_logic = df["GEval_Logic"].mean()

  new_row = {"model_name": pure_file_name, "average_nli": 1-average_nli, "average_readability": average_readability, "average_logic": average_logic}
  df2 = pd.concat([df2, pd.DataFrame([new_row])], ignore_index=True)


2it [00:00,  5.35it/s]

[Document(page_content="of sounds like I'm asking them-- ### Human: Do you think you could answer that for me? ### Assistant: --am I? And when people ask me my age, I sort of go, 23rd of January, I am-- and I sort of compare myself to my-- ### Human: Because I think-- ### Assistant: --other-- yeah. I must be-- ### Human: --because I saw you-- ### Assistant: --14. I don't. ### Human: --when you were 14. Was that-- ### Assistant: I must be-- ### Human: --before your birthday? ### Assistant: Wow. I am-- ### Human: You're", metadata={'source': '/content/drive/MyDrive/Colab Notebooks/result/RAG_362.txt'}), Document(page_content="in year 10, right? ### Assistant: Yeah. I'm in year 10, and I'm a year younger-- ### Human: So you'd be 15. ### Assistant: --than my age. Yeah. I would be. I was 14-- yes. I am 15. ### Human: You are 15? ### Assistant: All right. ### Human: You figured out your age? ### Assistant: Now, we've got that part down. I am 15 because I know that I'm-- ### Human: So you wer

4it [00:00,  5.93it/s]

[Document(page_content='Assistant: hours. The past week and a half, nothing\'s been really on my mind. So I can sleep. Wake up the next day. Go back to school. I\'ve been enjoying school a lot more. ### Human: That\'s a huge difference because even two months ago you were like, "I\'m not really enjoying it." What\'s made school-- ### Assistant: I think the fact that I\'m now-- next year I\'m doing it. ### Human: Wow. What\'s made that decision ?-- ### Assistant: I guess the fact that I-- ### Human: --for next year? ###', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/result/RAG_362.txt'}), Document(page_content="Assistant: --don't like the whole idea of going to university or just doing-- I sort of learn a lot more practically. So if you put something in front of me, and it was a drawing or something like that, I'd more likely be able to understand that. If you just wrote it on the board or something like that. And I've sort of picked things up by watching people do them o

6it [00:01,  6.29it/s]

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

29it [01:52,  3.88s/it]

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

30it [01:56,  3.88s/it]


In [ ]:
df2.to_csv(f"/content/drive/MyDrive/Colab Notebooks/result/result2.csv", index=False)

In [ ]:
df2

,model_name,average_nli,average_readability,average_logic
0,kto-0.4,0.203133,0.855757,0.828366
1,gpt_rag,0.496389,0.929989,0.965947
2,gpt,0.424993,0.940139,0.991434
3,falcon-finetuned,0.314459,0.233821,0.260801
4,dpo-0.4,0.221491,0.843937,0.852779
5,falcon,0.320680,0.716404,0.729864
6,dpo-1.0,0.177933,0.800139,0.785722
7,dpo-0.7,0.293748,0.790270,0.840389
8,falcon-rag,0.530631,0.813640,0.562383
9,dpo-0.1,0.206013,0.802300,0.832610
